# Convolutional Neural Network (CNN) for MNIST classification

In [1]:
import numpy as np
import torch
import time
import platform

In [2]:
print(f'Pytorch version: {torch.__version__}')
print(f'cuda version: {torch.version.cuda}')
print(f'Python version: {platform.python_version()}')

Pytorch version: 1.0.0
cuda version: 9.0.176
Python version: 3.6.8


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## MNIST Data preparation

### Load MNIST data and normalize

In [4]:
from MNISTtools import load, show

In [5]:
xtrain, ltrain = load(dataset='training', path='dataset/')
xtest, ltest = load(dataset='testing', path='dataset/')

In [6]:
def normalize_MNIST_images(x):
    '''
    Args:
        x: data
    '''
    x_norm = x.astype(np.float32)
    return x_norm*2/255-1

In [7]:
# normalization
xtrain = normalize_MNIST_images(xtrain)
xtest = normalize_MNIST_images(xtest)

### Reshape
Torch expects that the input of a convolutional layer is stored in the following format
`Batch size × Number of input channels × Image width × Image height`

In [8]:
# reshape to 3d
xtrain = xtrain.reshape([28,28,-1])[:,:,None,:]
xtest = xtest.reshape([28,28,-1])[:,:,None,:]
print(f'shape of xtrain after reshape is {xtrain.shape}.')
print(f'shape of xtest after reshape is {xtest.shape}.')

shape of xtrain after reshape is (28, 28, 1, 60000).
shape of xtest after reshape is (28, 28, 1, 10000).


In [9]:
# moveaxis
xtrain = np.moveaxis(xtrain, (2,3), (1,0))
xtest = np.moveaxis(xtest, (2,3), (1,0))
print(f'shape of xtrain after moveaxis is {xtrain.shape}.')
print(f'shape of xtest after moveaxis is {xtest.shape}.')

shape of xtrain after moveaxis is (60000, 1, 28, 28).
shape of xtest after moveaxis is (10000, 1, 28, 28).


### Wrap all the data into torch Tensor

In [10]:
xtrain = torch.from_numpy(xtrain)
ltrain = torch.from_numpy(ltrain)
xtest = torch.from_numpy(xtest)
ltest = torch.from_numpy(ltest)

In [11]:
xtrain_gpu = xtrain.to(device)
ltrain_gpu = ltrain.to(device)
xtest_gpu = xtest.to(device)
ltest_gpu = ltest.to(device)

---
## LeNet -5 network

* Convolutional layers can be created as `nn.Conv2d(N, C, K)`. For input images of size `W×H`, the output feature maps have size `[W−K+1]x[H−K+1]`.  

* Maxpooling is implemented like any other non-linear function (such as ReLU or softmax). For input images of size `W×H`, the output feature maps have size `[W/L]×[H/L]`.  

* A fully connected layer can be created as `nn.Linear(M, N)`.

Architecture:  

(a) a convolutional layer connecting the input image to `6` feature maps with `5×5` convolutions (`K=5`) and followed by ReLU and maxpooling (`L=2`)  

(b) a convolutional layer connecting the `6` input channels to `16` output channels with `5×5` convolutions and followed by ReLU and maxpooling (`L=2`)  

(c) a fully-connected layer connecting `16` feature maps to `120` output units and followed by ReLU  

(d) a fully-connected layer connecting `120` inputs to `84` output units and followed by ReLU  

(e) a final linear layer connecting `84` inputs to `10` linear outputs (one for each of our digits)

First layer  
* input: `(28, 28, 1)`  
* after *padding*: `(32, 32, 1)`
* after convolution(kernel=`5x5`): `(28, 28, 6)` where `28=32-5+1`  
* after ReLU: `(28, 28, 6)`  
* after maxpooling(stride=`2x2`): `(14, 14, 6)` $\Rightarrow$ **OUTPUT**  


Second layer
* input: `(14, 14, 6)`
* after convolution(kernel=`5x5`): `(10, 10, 16)`
* after ReLU: `(10, 10, 16)`  
* after maxpooling(stride=`2x2`): `(5, 5, 16)` $\Rightarrow$ **OUTPUT**  


Third layer
* input: `(5, 5, 16)` $\Rightarrow$ `5x5x16=400`  
* after fully-connected: `(120, 1)`
* after ReLU: `(120, 1)` $\Rightarrow$ **OUTPUT**  


Fourth layer
* input: `(120, 1)`
* after fully-connected: `(84, 1)`
* after ReLU: `(84, 1)` $\Rightarrow$ **OUTPUT**  


Fifth layer
* input: `(84, 1)`
* after fully-connected: `(10, 1)`
* after ReLU: `(10, 1)` $\Rightarrow$ **OUTPUT**  

In [12]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class LeNet(nn.Module):

    # network structure
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        '''
        Get the number of features in a batch of tensors `x`.
        '''
        size = x.size()[1:]
        return np.prod(size)

### Check the network structure

In [14]:
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Check the network parameters

In [15]:
for name, param in net.named_parameters():
    print(name, param.size(), param.requires_grad)

conv1.weight torch.Size([6, 1, 5, 5]) True
conv1.bias torch.Size([6]) True
conv2.weight torch.Size([16, 6, 5, 5]) True
conv2.bias torch.Size([16]) True
fc1.weight torch.Size([120, 400]) True
fc1.bias torch.Size([120]) True
fc2.weight torch.Size([84, 120]) True
fc2.bias torch.Size([84]) True
fc3.weight torch.Size([10, 84]) True
fc3.bias torch.Size([10]) True


### The accuracy without backprop

In [16]:
# avoid tracking for gradient during testing and then save some computation time
with torch.no_grad():
    yinit = net(xtest)

In [17]:
_, lpred = yinit.max(1)
print(100 * (ltest == lpred).float().mean())

tensor(8.9300)


`ltest == lpred` generates a tensor with values of `0` and `1`, where `0` means inequal and `1` means equal. Therefore, `(ltest == lpred).float().mean()` implies the accuracy which is **11.55%**.

### (Mini-Batch) Stochastic Gradient Descent (SGD) with cross-entropy and momentum

**Note**: PyTorch’s CrossEntropyLoss is the composition of a softmax activation with the standard cross-entropy loss.

In [18]:
def backprop_deep(xtrain, ltrain, net, T, B=100, gamma=.001, rho=.9):
    '''
    Backprop.
    
    Args:
        xtrain: training samples
        ltrain: testing samples
        net: neural network
        T: number of epochs
        B: minibatch size
        gamma: step size
        rho: momentum
    '''
    N = xtrain.size()[0]     # Training set size
    NB = N//B                # Number of minibatches
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=gamma, momentum=rho)
    
    for epoch in range(T):
        running_loss = 0.0
        shuffled_indices = np.random.permutation(NB)
        for k in range(NB):
            # Extract k-th minibatch from xtrain and ltrain
            minibatch_indices = range(shuffled_indices[k]*B, (shuffled_indices[k]+1)*B)
            inputs = xtrain[minibatch_indices]
            labels = ltrain[minibatch_indices]

            # Initialize the gradients to zero
            optimizer.zero_grad()

            # Forward propagation
            outputs = net(inputs)

            # Error evaluation
            loss = criterion(outputs, labels)

            # Back propagation
            loss.backward()

            # Parameter update
            optimizer.step()

            # Print averaged loss per minibatch every 100 mini-batches
            # Compute and print statistics
            with torch.no_grad():
                running_loss += loss.item()
            if k % 100 == 99:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, k + 1, running_loss / 100))
                running_loss = 0.0

In [19]:
net = LeNet()

In [20]:
start = time.time()
backprop_deep(xtrain, ltrain, net, T=3)
end = time.time()
print(f'It takes {end-start:.6f} seconds.')

[1,   100] loss: 2.301
[1,   200] loss: 2.291
[1,   300] loss: 2.277
[1,   400] loss: 2.250
[1,   500] loss: 2.174
[1,   600] loss: 1.864
[2,   100] loss: 1.022
[2,   200] loss: 0.640
[2,   300] loss: 0.535
[2,   400] loss: 0.435
[2,   500] loss: 0.395
[2,   600] loss: 0.341
[3,   100] loss: 0.293
[3,   200] loss: 0.293
[3,   300] loss: 0.260
[3,   400] loss: 0.242
[3,   500] loss: 0.227
[3,   600] loss: 0.210
It takes 36.121122 seconds.


### Evaluate on the testing dataset (CPU)

In [21]:
start = time.time()
backprop_deep(xtest, ltest, net, T=3)
end = time.time()
print(f'It takes {end-start:.6f} seconds.')

[1,   100] loss: 0.182
[2,   100] loss: 0.166
[3,   100] loss: 0.151
It takes 5.754438 seconds.


In [22]:
y = net(xtest)

In [23]:
print(100 * (ltest==y.max(1)[1]).float().mean())

tensor(95.4200)


The accuracy for 3 epochs is **95.42%**.

### Network on GPU

In [24]:
net_gpu = LeNet().to(device)

In [25]:
start = time.time()
backprop_deep(xtrain_gpu, ltrain_gpu, net_gpu, T=10)
end = time.time()
print(f'It takes {end-start:.6f} seconds.')

[1,   100] loss: 2.298
[1,   200] loss: 2.288
[1,   300] loss: 2.274
[1,   400] loss: 2.253
[1,   500] loss: 2.201
[1,   600] loss: 2.031
[2,   100] loss: 1.437
[2,   200] loss: 0.852
[2,   300] loss: 0.606
[2,   400] loss: 0.470
[2,   500] loss: 0.402
[2,   600] loss: 0.357
[3,   100] loss: 0.311
[3,   200] loss: 0.294
[3,   300] loss: 0.251
[3,   400] loss: 0.219
[3,   500] loss: 0.227
[3,   600] loss: 0.210
[4,   100] loss: 0.194
[4,   200] loss: 0.181
[4,   300] loss: 0.164
[4,   400] loss: 0.168
[4,   500] loss: 0.173
[4,   600] loss: 0.151
[5,   100] loss: 0.143
[5,   200] loss: 0.142
[5,   300] loss: 0.131
[5,   400] loss: 0.136
[5,   500] loss: 0.127
[5,   600] loss: 0.115
[6,   100] loss: 0.116
[6,   200] loss: 0.122
[6,   300] loss: 0.110
[6,   400] loss: 0.105
[6,   500] loss: 0.106
[6,   600] loss: 0.104
[7,   100] loss: 0.111
[7,   200] loss: 0.094
[7,   300] loss: 0.097
[7,   400] loss: 0.092
[7,   500] loss: 0.097
[7,   600] loss: 0.087
[8,   100] loss: 0.081
[8,   200] 

### Re-evaluate on the testing dataset (GPU)

In [26]:
y = net_gpu(xtest_gpu)

In [27]:
print(100 * (ltest==y.max(1)[1].cpu()).float().mean())

tensor(97.9800)


The accuracy for 10 epochs is **97.98%** which is higher than that for 3 epochs, **95.42%**.

In [28]:
start = time.time()
backprop_deep(xtest_gpu, ltest_gpu, net_gpu, T=10)
end = time.time()
print(f'It takes {end-start:.6f} seconds.')

[1,   100] loss: 0.063
[2,   100] loss: 0.059
[3,   100] loss: 0.052
[4,   100] loss: 0.050
[5,   100] loss: 0.049
[6,   100] loss: 0.045
[7,   100] loss: 0.046
[8,   100] loss: 0.043
[9,   100] loss: 0.042
[10,   100] loss: 0.038
It takes 2.311700 seconds.
